In [18]:
import os
from types import SimpleNamespace
from typing import (List, Dict, Any, Optional, Callable, Type, Tuple, # pylint: disable=unused-import
                    AsyncIterator, Union, TypeAlias) 
from pathlib import Path

import header
from header import __root__
from src import gs
from src.webdriver.llm_driver import Driver 
from src.utils.file import get_filenames_from_directory
from src.utils.jjson import j_loads, j_loads_ns
from src.utils.printer import pprint as print

2025-04-28 03:04:39,680 - INFO - ℹ️ Асинхронные контроллеры успешно импортированы из src.webdriver.llm_driver.controllers. 
2025-04-28 03:04:39,690 - WARNING - ⚠️ .env файл не найден по пути: C:\Users\user\Documents\repos\hypotez\.env. 
2025-04-28 03:04:39,691 - INFO - ℹ️ Конфигурация успешно загружена из C:\Users\user\Documents\repos\hypotez\src\webdriver\llm_driver\use_llm.json 
2025-04-28 03:04:39,692 - DEBUG - 🐛 GEMINI_API_KEY из gs 
2025-04-28 03:04:39,692 - DEBUG - 🐛 OPENAI_API_KEY из gs 
2025-04-28 03:04:39,692 - DEBUG - 🐛 SERPAPI_API_KEY из gs 
2025-04-28 03:04:39,693 - INFO - ℹ️ Config Gemini: Status=active, Model=gemini-2.5-flash-preview-04-17, Key Present=True 
2025-04-28 03:04:39,693 - INFO - ℹ️ Config OpenAI: Status=disabled, Model=gpt-4o, Key Present=True 
2025-04-28 03:04:39,694 - INFO - ℹ️ Config SerpAPI: Key Present=True 
2025-04-28 03:04:41,021 - INFO - Anonymized telemetry enabled. See https://docs.browser-use.com/development/telemetry for more information.


In [19]:

class Config:
    ENDPOINT:Path = __root__/'SANDBOX'/'davidka'
    driver:Driver = Driver()

2025-04-28 03:04:48,205 - INFO - ℹ️ --- Начало СИНХРОННОЙ инициализации Driver --- 
2025-04-28 03:04:48,220 - WARNING - ⚠️ OpenAI LLM не инициализирован (Key=True, Status=disabled, Model=gpt-4o) 
2025-04-28 03:04:48,221 - INFO - ℹ️ Инициализация Gemini: Model=gemini-2.5-flash-preview-04-17 
2025-04-28 03:04:48,271 - INFO - ℹ️ Gemini LLM инициализирован. 
2025-04-28 03:04:48,273 - INFO - ℹ️ --- Синхронная инициализация Driver завершена. Вызовите async_init() --- 


In [20]:
crawl_files_list:list = get_filenames_from_directory(Config.ENDPOINT/'random_urls')

In [6]:
def get_products_urls_list_from_files(crawl_files_list:list = []) -> list:
    """
   Функция читает содержимое файлов  в директории `mining_data`, перемешивает их и возвращает одним большим списком
   """
    products_urls_list = []
    for filename in crawl_files_list or Config.crawl_files_list:
        try:
            file_path = Config.mining_data_path / filename
            crawl_data = j_loads(file_path)['products']
            for product in crawl_data:
                products_urls_list.append(product['product_url'])
        except Exception as ex:
            logger.error(f'Ошибка при обработке файла/n {filename=}/n', ex)
            ...
    random.shuffle(products_urls_list)
    return products_urls_list if isinstance(products_urls_list, list) else [products_urls_list]

def yield_product_urls_from_files(directory: Path = Config.mining_data_path, pattern: str = 'json'):
    """
    Функция возвращает генератор списка `url` Применяется на больших объемах данных
    """
    filenames = get_filenames_from_directory(directory, pattern)
    for filename in filenames:
        try:
            file_path = directory / filename
            crawl_data = j_loads(file_path)['products']
            for product in crawl_data:
                yield product['product_url']
        except Exception as ex:
            logger.error(f'Ошибка при обработке файла {filename=}', ex)
            ...

def get_categories(crawl_files_list:list = []) -> list:
    """Возвращает все категории из файлов словарей для майнинга"""
    categories_list:list = []
    for filename in crawl_files_list or Config.crawl_files_list:
        try:
            file_path = Config.mining_data_path / filename
            crawl_data = j_loads(file_path)['products']
            for product in crawl_data:
                try:
                    categories_list.append(product['parent_category'])
                except:
                    ...
                try:    
                    categories_list.append(product['category_name'])
                except:
                    ...
        except Exception as ex:
            logger.error(f'Ошибка при обработке файла/n {filename=}/n', ex)
            ...
            return categories_list
            ...
    categories_list = list(set(categories_list))
    random.shuffle(categories_list)
    return categories_list 

async def get_products_by_category(category:str, num_of_links:str = '5'):
        try:
            driver:SimpleDriver = Config.driver
            logger.info(f'Обработка {category=}')
            task = Config.generate_product_links_instruction.replace('<product category name>', category).replace('<num of links>', num_of_links)
            extracted_data = await driver.simple_process_task_async(task)
            print('EXTRACTED DATA')
            print(extracted_data['history']['AgentHistory'])
            ...
            return extracted_data
        except Exception as ex:
            logger.error(f'Ошибка при обработке {category=}', ex)
            ...
#print(data_files_list)